# Считывание данных


In [ ]:
import pandas as pd
import matplotlib as plt 
import random
import math
import time
data = pd.read_csv("docking_data.csv")
data

,E_fst,E_fst_elstat,E_fst_VdW,E_fst_SP,group_size,type_of_complex_in_group,E_snd_SP,avg_E_fst,avg_E_snd_SP,E_third,alt_E_third,E_third_SP,avg_E_third,avg_alt_E_third,avg_E_third_SP,is_real
0,6206.36,2497.87,17855.28,-962.05,22,middle,-714.59,6392.68,-887.89,-7804.31,-68.80,-690.03,-6821.58,-71.05,-804.82,No
1,7014.94,2731.57,20616.55,-1120.45,15,border,-876.50,6734.31,-792.75,-7660.53,-145.94,-756.74,-6208.02,-71.29,-712.62,No
2,7085.45,2737.57,21195.77,-761.06,14,border,-624.35,6928.07,-603.86,-7509.69,-94.86,-475.83,-7071.98,-85.49,-479.08,No
3,6148.76,4068.35,9399.78,-1403.11,18,border,-1071.77,6361.60,-925.44,-7481.41,-26.12,-732.11,-6198.65,-11.64,-701.10,No
4,6556.68,4008.87,11457.76,-1793.80,14,border,-1421.39,6434.94,-1405.50,-7469.59,-56.95,-1226.14,59181060.00,59168384.00,-1347.05,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31456,3912.76,1621.84,11302.28,-213.30,19,centroid,-149.69,4037.06,-112.07,-1009.06,-38.06,-91.34,-745.62,-33.36,-51.76,No
31457,4266.37,2878.17,6698.90,-338.95,30,middle,-238.18,4077.41,-290.96,-913.20,-54.74,-208.98,-895.96,-55.76,-243.12,No
31458,3741.59,2195.55,7262.18,-655.27,23,middle,-461.86,3921.64,-418.57,-819.98,-16.04,-423.07,-819.98,-16.04,-423.07,No
31459,3995.26,2517.48,7173.27,-301.84,19,middle,-245.14,3975.31,-216.60,-564.29,-43.46,-210.81,141.76,128.86,-206.33,No


Закодируем категориальные признаки


In [ ]:
for ind in data.index:
  if data['is_real'][ind] == "No":
    data['is_real'][ind] = 0
  else:
    data['is_real'][ind] = 1


Разделение данных на train и test

In [ ]:
train = data.sample(frac = 0.7)
test = pd.concat([data,train]).drop_duplicates(keep=False) #получаем разницу между data и train

#Преобразуем в список
data_as_train = []
data_as_test = []

for (_, row) in train.iterrows():
  data_as_train.append(row) 
for i in range (len(data_as_train)):
  data_as_train[i]=i, data_as_train[i] 

for (_, row) in test.iterrows():
  data_as_test.append(row) 
for i in range (len(data_as_test)):
  data_as_test[i]=i, data_as_test[i] 

## Линейное решение
Классифицируем данные, выбрав один параметр и отсекая по константному значению

In [ ]:
a=-1000
def f(factors_of_complex):

    if factors_of_complex["E_fst"] < a:
      return 1
    else: 
      return 0
      
print(f(data_as_test[0][1]))

0


## Логистическая регрессия

In [ ]:
def logistic_regression(factors_of_complex,factors_list, max_list ,weights_list):
  sinusoid = 0
  
  for i in range(len(max_list)):

    sinusoid += weights_list[i]*(factors_of_complex[factors_list[i]] / max_list[i])
  
  sinusoid += weights_list[len(weights_list) - 1]
  if pow((1+math.exp(-1*sinusoid)), -1) > 0.5:
    return 1
  else: 
    return 0


#Оценка качества











In [ ]:
def accuracy (data):
  T = 0
  F = 0
  for i in range (len(data)):
      target = data[i][1]["is_real"]
      if target == f(data[i][1]):
        T += 1
      else:
        F += 1
  return T/(T + F) #считаем точность функции 

In [ ]:
def f1score (data_as_train, factors_list, max_list ,weights_list):
 
  TP=0
  FP=0
  FN=0
  TN=0
  for i in range(len(data_as_train)):  #смотрим количество совпадений с целевым признаком
      target = data_as_train[i][1]["is_real"]
      result = logistic_regression(data_as_train[i][1], factors_list, max_list ,weights_list)

      if target == 1 and result == 1:
        TP += 1
      if target == 0 and result == 1:
        FP += 1
      if target == 1 and result == 0:
        FN += 1
      if target == 0 and result == 0:
        TN += 1
  return (TP)/(TP+0.5*(FP+FN))#считаем точность функции


#Обучение логистической регрессии

Напишем функцию, возвращающую максимальное по модулю значение по одному признаку

In [ ]:
def max_factor(data, factor_name): #МАКС ЗНАЧЕНИЕ
  max = -1
  for i in range (len(data)):
    if abs(data[i][1][factor_name])>max:
      max = abs(data[i][1][factor_name])
      max1 = data[i][1][factor_name]
  return max1 

Напишем функцию, которая будет лучше подбирать веса: для определенного значения будем смотреть изменения f1_score при приращении параметра на step

In [ ]:
def better_weight(current_weights, index_of_better, step):

  weight = current_weights[index_of_better]

  weight_right = weight + step
  weight_left = weight - step

  current_weights[index_of_better] = weight_right
  current_weights[index_of_better] = weight_left

  f1_right = f1score(data_as_train, factors_list, max_list ,weights_list)
  f1_left = f1score(data_as_train, factors_list, max_list ,weights_list)

  if f1_right >= f1_left:
    current_weights[index_of_better] = weight_right
    return current_weights[index_of_better]
  else: 
    current_weights[index_of_better] = weight_left
    return  current_weights[index_of_better]

Оптимизируем подбор параметров, используя стохастический метод - имитацию отжига

In [ ]:
def simulated_annealing(scale_step, temp_step, start_temp, end_temp, data_as_train, factors_list, max_list, weights_list):
  prev_score = f1score(data_as_train,factors_list, max_list, weights_list)

  T = start_temp #начальная температура
  while (T > end_temp):

      prev_weight = weights_list.copy()
      copy_f1score =  prev_score
      random_weight_num = random.randint(0,7)
      weights_list[random_weight_num] = better_weight(weights_list, random_weight_num, random.random()*scale_step)
      
      T *= temp_step #понижаем температуру
      
      print(T,'\n')
      print(weights_list)

      #Обновляем качество
      f1 = f1score(data_as_train,factors_list, max_list ,weights_list)
      desicion = math.exp(1000 * (f1 - copy_f1score) / T)
      if ((f1 - copy_f1score) > 0 | (random.random() <= desicion)):
        prev_score = f1score(data_as_train,factors_list, max_list ,weights_list)
        best_weights = weights_list

        print(math.exp(1000*(f1 - copy_f1score) / T)) 
      else:
        weights_list = prev_weight.copy()
      print(prev_score,'\n')
  return best_weights

a = 29.48389832287498
b = 56.316526694654584
c = 1.1009480363451694
d = -17.54300736313281
e = -3.489668353493901
f = 8.574042399095756
g = -4.116129333039096
h = 20.565020511662908    
beta = -30.909232430870986

weights_list = [a, b, c, d, e, f, g, h, beta]
factors_list = ["E_fst","group_size","E_fst_VdW","E_fst_SP","avg_E_fst","avg_E_snd_SP","E_third","alt_E_third"] 

max_list = []
for factor in factors_list:
  max_list.append(max_factor(data_as_train, factor))

result = simulated_annealing(5, 0.9, 90000, 0.01, data_as_train, factors_list, max_list, weights_list)
print(result)

# ТЕСТИРОВАНИЕ


In [ ]:
#ТЕСТИРОВАНИЕ
a=28.117782858612504
b=53.80753829268679
c=3.2551306977776284 
d=-12.796445064951808
e=-5.269774492798852
f=6.0999941893402925
g=-6.699565037075078
h=25.454449083705306
beta=-30.909232430870986
print(data_as_test[0][1]["is_real"])
f1score(data_as_test, factors_list, max_list ,weights_list)

0


0.33634992458521873

In [ ]:
def gain_counter(data_as_train, alpha, Sum_x_node,xnode0, xnode1,length):
    x_right_node0=0
    x_right_node1=0
    x_left_node0=0
    x_left_node1=0
    for j in range (length):
                if data_as_train[j][0]<alpha:

                  if data_as_train[j][1]=="Yes":
                    x_left_node1+=1
                  if data_as_train[j][1]=="No":
                    x_left_node0+=1
                else:

                  if data_as_train[j][1]=="Yes":
                    x_right_node1+=1
                  if data_as_train[j][1]=="No":
                    x_right_node0+=1
          # -----------
    Sum_left=x_left_node0+x_left_node1 
    Sum_right=x_right_node0+x_right_node1
    I_x_node = 2*(xnode1)*(xnode0)/(Sum_x_node*Sum_x_node)
    inf = float('inf')
    
    if Sum_left==0:
      I_x_left=inf
      I_x_right = 2*(x_right_node1)*(x_right_node0)/(Sum_right*Sum_right)
    elif Sum_right==0:
      I_x_right=inf
      I_x_left = 2*(x_left_node1)*(x_left_node0)/(Sum_left*Sum_left)
    else:
      I_x_left = 2*(x_left_node1)*(x_left_node0)/(Sum_left*Sum_left)
      I_x_right = 2*(x_right_node1)*(x_right_node0)/(Sum_right*Sum_right)
    # ----------
    INFO_GAIN =I_x_node-(Sum_left/Sum_x_node)*I_x_left - (Sum_right/Sum_x_node)*I_x_right
    return INFO_GAIN

In [ ]:
def recurtion_squares(T, data_set,test_listman,T00,Sum_x_node,xnode0, xnode1):
  #global keke
  T0=T
  data_set1= []
  data_set2= []
  
 
  if (T<=2):
    alpha1 = data_set[T-1][0]
    alpha2 = data_set[T-2][0]
    if alpha1>alpha2:
      return alpha1
    else:
      return alpha2
   
  T = int(T/2)
  
  alpha1 = data_set[T][0]
  alpha2 = data_set[T-2][0]

  for j in range (T):
    data_set1.append(data_set[j])

  for m in range (T, T0):
    data_set2.append(data_set[m])

  length1 = len(data_set1)
  
  info_gain1 = gain_counter(test_listman, alpha1, Sum_x_node,xnode0, xnode1,T00)

  length2 = len(data_set2)
  info_gain2 = gain_counter(test_listman, alpha2, Sum_x_node,xnode0, xnode1,T00)
   # выход из рекурсии
 
  if info_gain1<info_gain2:
    return recurtion_squares(T, data_set1,test_listman,T00,Sum_x_node,xnode0, xnode1)
  else: 
    return recurtion_squares(T0-T, data_set2,test_listman,T00,Sum_x_node,xnode0, xnode1)

In [ ]:
test_listman = []

for i in range (1000):
   test_listman.append([i, "Yes"])
for i in range (1000,2000):
   test_listman.append([i, "No"])
xnode0=0
xnode1=0
T00=2000
for i in range (2000) :
  emae = test_listman[i][1]
  if emae=="Yes":
      xnode1+=1
  if emae=="No":
      xnode0+=1
Sum_x_node = xnode1+xnode0
recurtion_squares(2000, test_listman,test_listman,T00)

# Дерево принятий

In [ ]:
listishe_of_factors = ["E_fst","E_fst_elstat","E_fst_VdW","E_fst_SP","group_size","E_snd_SP","avg_E_fst","avg_E_snd_SP","E_third","alt_E_third","E_third_SP","avg_E_third","avg_alt_E_third", "avg_E_third_SP"]
class Node:

  def __init__(self,listishe_of_factors):
    self.listishe_of_factors = listishe_of_factors
    self.factor_name = None
    self.alpha = None
    self.factors_list = random.choices(listishe_of_factors,k=3)
    self.left = None
    self.right = None
    self.is_real = None
    self.elements=[] 
    
    
  
  def solution (self, data_as_train): # solution предсказывает класс для элемента
   
   if self.is_real != None:
      return self.is_real
   else:
      if data_as_train[self.factor_name]<self.alpha:
          return self.left.solution(data_as_train)
      else:
          return self.right.solution(data_as_train)
  #если условие выполнено, то делегируем принять решение левому ребенку, если нет- то правому
  
  
  def build(self, data_as_train,counter):
    counter+=1
    data_right = []
    data_left = []
    xnode1=0
    xnode0=0
    length = len(data_as_train)
   
    for i in range (length) :
      
      emae = data_as_train[i][1]["is_real"]
      if emae=="Yes":
        xnode1+=1
      if emae=="No":
        xnode0+=1
       
    if xnode0==0:
      self.is_real = 1
      return
    elif xnode1==0:
      self.is_real = 0
      return
    
    
    
    if counter>25:
      if xnode1>=xnode0:
        self.is_real=1
        return None
      else: 
        self.is_real=0
        return None
    
    Sum_x_node =xnode0+ xnode1
    
    last_gain=-10000
    max_factor = "E_fst"
    maximum_alpha=0
# РЕАЛИЗУЕМ МЕТОД ДИХОТОМИИ ДЛЯ ПОИСКА АЛЬФЫ!!!!!!!
    for factor_name in self.factors_list:
     
      sorted_list = [] 
      for q in range (length):
        sorted_list.append([data_as_train[q][1][factor_name], data_as_train[q][1]["is_real"]])
      sorted_list.sort(key=lambda sorted_list:sorted_list[0])
      keke = sorted_list.copy()
     
      T00 = length
      T=length
      best_alpha = recurtion_squares(T, sorted_list,sorted_list,T00,Sum_x_node,xnode0, xnode1)    
      max_gain=gain_counter(sorted_list, best_alpha,Sum_x_node, xnode0, xnode1,length)
            
            
      if(max_gain>last_gain):
        max_factor = factor_name
        last_gain = max_gain
        maximum_alpha = best_alpha
      
     
     
    for j in range (length):
            if data_as_train[j][1][max_factor]<maximum_alpha:
              data_left.append(data_as_train[j])
              
            else:
              data_right.append(data_as_train[j])
    self.factor_name = max_factor
    self.alpha=maximum_alpha    
    
    # перебираем все factor_name и alpha так, чтобы 
    # IG при разбиении elements_factors по правилу x[F]< alpha был максимальным
    # 2. Присваиваем нашим именам факторов и альфа найденные, присваиваем elements elements_factors_train
    # 3. self.left = Node() self.right=Node()
    # Запускаем построение left right на подмножествах elements_factors_train, с помощью разбиения по новому правилу
    
    self.left = Node(listishe_of_factors)
    self.right = Node(listishe_of_factors)
    self.left.build(data_left,counter)
    self.right.build(data_right,counter)


# Выделение для подбора весов

In [ ]:
length1 =int(0.2*len(data_as_train))
test_as_train = random.choices(data_as_train, k=length1) #рандомно выбираем 10% тестовых данных
train_as_train = []
m=0
for (i, row) in data_as_train:
  k=i
  k1=i
  for (j, _) in test_as_train:
    if i==j:
      k1+=1
  if k1==k:
    train_as_train.append((i, row)) #закидываем в трэйн остальные 90%

# Постройка леса 


In [ ]:
length =len(train_as_train)
tree_list= []
for j in range (200):
        print (j)
        tree_data_as_train = random.choices(train_as_train, k=length) 
        tree = Node(listishe_of_factors)
        tree.build(tree_data_as_train, 0)
        tree_list.append(tree)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: ignored

In [ ]:
length = len(test_as_train)
TP=0
FP=0
FN=1
TN=0
max_f1 = (TP)/(TP+0.5*(FP+FN))
max_a=0
max_b=0
T=900
a = 1
b = 1
while T>0.1:
  T=T*0.9
  print(T)
  last_f1_trying=max_f1
  copy_a = max_a
  copy_b = max_b
  a = random.randint(1,30)
  b = random.randint(1,30)
  TP=0
  FP=0
  FN=0
  TN=0
  for i in range (length):
        cow0=0
        cow1=0
        for j in range (200):
          if tree_list[j].solution(test_as_train[i][1]) == 1:
            cow1+=a
          else:
            cow0+=b
        if cow1>cow0:
          treeshka=1
        else:
          treeshka=0
        emae = test_as_train[i][1]["is_real"]
        if emae=="Yes" and treeshka==1:
          TP+=1
        if emae=="No" and treeshka==1:
          FP+=1
        if emae=="Yes" and treeshka==0:
          FN+=1
        if emae=="No" and treeshka==0:
          TN+=1
  f1_trying=(TP)/(TP+0.5*(FP+FN))
  if f1_trying>last_f1_trying:
    max_f1 = f1_trying
    max_a = a
    max_b=b
  elif random.random() <= ( math.exp(1000 * (f1_trying - last_f1_trying) / T)):
    max_f1 = f1_trying
    max_a = a
    max_b=b
  elif last_f1_trying == max_f1:
    max_f1 = last_f1_trying
    max_a = copy_a
    max_b = copy_b
  print(max_f1)
print (max_a, max_b)

810.0
0.13947696139476962
729.0
0.3298227587978423
656.1
0.18094089264173704
590.49
0.3184183142559833
531.441
0.3184183142559833
478.29690000000005
0.0
430.4672100000001
0.018867924528301886
387.4204890000001
0.09511568123393316
348.6784401000001
0.3019271948608137
313.8105960900001
0.07561929595827901
282.4295364810001
0.13947696139476962
254.18658283290011
0.016194331983805668
228.76792454961011
0.0
205.8911320946491
0.2773837667454689
185.3020188851842
0.46162790697674416
166.7718169966658
0.46162790697674416
150.09463529699923
0.46162790697674416
135.0851717672993
0.46162790697674416
121.57665459056938
0.32854209445585214
109.41898913151245
0.32854209445585214
98.4770902183612
0.32854209445585214
88.62938119652507
0.32854209445585214
79.76644307687256
0.32854209445585214
71.78979876918531
0.4013709063214014
64.61081889226678
0.4013709063214014
58.1497370030401
0.4013709063214014
52.33476330273609
0.4291497975708502
47.10128697246248
0.4654459539279386
42.391158275216235
0.46544595

# Лес решений 11 6

In [ ]:
def f1score_tree(data_as_train,max_a,max_b):
  length = len(data_as_train)
  TP=0
  FP=0
  FN=0
  TN=0

  for i in range (length):
      cow0=0
      cow1=0
      for j in range (200):
        if tree_list[j].solution(data_as_train[i][1]) == 1:
          cow1+=max_a
        else:
          cow0+=max_b
      if cow1>cow0:
        treeshka=1
      else:
        treeshka=0
      emae = data_as_train[i][1]["is_real"]
      if emae=="Yes" and treeshka==1:
        TP+=1
      if emae=="No" and treeshka==1:
        FP+=1
      if emae=="Yes" and treeshka==0:
        FN+=1
      if emae=="No" and treeshka==0:
        TN+=1
  return TP, FP, FN, TN, (TP)/(TP+0.5*(FP+FN)), ((TP+TN)/(TP+TN+FP+FN)) #считаем точность функции

In [ ]:
f1score_tree(test,max_a,max_b)

(549, 846, 440, 4457, 0.46057046979865773, 0.7956134774316592)

In [ ]:
a = [[1,2], [1,2]]

a.append([1 , 2])
a[0][1]

2